In [2]:
import argparse
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
import os
import types

In [3]:
# 🔧 Argümanlar
args = types.SimpleNamespace()
args.model = r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\model_10.pth"
args.cuda = False

# 📁 Girdi ve çıktı klasörleri
val_data_dir = r"C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\val\Data"
output_dir = os.path.join(os.path.dirname(val_data_dir), "Predicted")
os.makedirs(output_dir, exist_ok=True)

# 🔄 Görseli tensöre çevirme fonksiyonu
img_to_tensor = transforms.ToTensor()

# 🖥️ Cihaz seçimi
device = torch.device("cuda:0" if (torch.cuda.is_available() and args.cuda) else "cpu")
print(f"Kullanılan cihaz: {device}")

# 🔁 Modeli yükle ve cihaza taşı
%run model.ipynb  
model = torch.load(args.model, map_location=device).to(device)
model.eval()

# 📂 Tüm görselleri sırayla işle
for fname in os.listdir(val_data_dir):
    if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    # 🔹 Giriş görüntüsünü aç
    img_path = os.path.join(val_data_dir, fname)
    img = Image.open(img_path).convert('YCbCr')
    y, cb, cr = img.split()

    # 🔹 Tensöre dönüştür
    input = img_to_tensor(y).unsqueeze(0).to(device)  # (1, 1, H, W)

    # 🔹 Model ile tahmin
    with torch.no_grad():
        out = model(input)

    # 🔹 Çıktıyı yeniden görsele çevir
    out = out.cpu()
    out_img_y = out[0].detach().numpy()
    out_img_y *= 255.0
    out_img_y = out_img_y.clip(0, 255)
    out_img_y = Image.fromarray(np.uint8(out_img_y[0]), mode='L')

    # 🔹 YCbCr birleştir ve RGB'ye çevir
    out_img = Image.merge('YCbCr', [out_img_y, cb, cr]).convert('RGB')

    # 💾 Kaydet
    output_path = os.path.join(output_dir, "pred_" + fname)
    out_img.save(output_path)
    print(f"✅ Kaydedildi: {output_path}")

Kullanılan cihaz: cpu
✅ Kaydedildi: C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\val\Predicted\pred_0.png
✅ Kaydedildi: C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\val\Predicted\pred_1.png
✅ Kaydedildi: C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\val\Predicted\pred_10.png
✅ Kaydedildi: C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\val\Predicted\pred_100.png
✅ Kaydedildi: C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\val\Predicted\pred_101.png
✅ Kaydedildi: C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\val\Predicted\pred_102.png
✅ Kaydedildi: C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\val\Predicted\pred_103.png
✅ Kaydedildi: C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\val\Predicted\pred_104.png
✅ Kaydedildi: C:\Users\melis\OneDrive\Masaüstü\image_processing_project\SRCNN-master\va

args = types.SimpleNamespace()
args.model = r"C:\Users\melis\OneDrive\Masaüstü\image processing project\SRCNN-master\model_10.pth"       # ✅ SRCNN model dosya adı
args.image = "image.jpg"        # ✅ Giriş görsel dosyası (YCbCr uyumlu)
args.zoom_factor = 2            # ✅ Yaklaştırma katsayısı (örnek: 2x)
args.cuda = False               # ✅ CUDA kullanılsın mı?

img = Image.open(args.image).convert('YCbCr')
img = img.resize(
    (int(img.size[0] * args.zoom_factor), int(img.size[1] * args.zoom_factor)),
    Image.BICUBIC
)
y, cb, cr = img.split()

# 🔄 Görseli tensöre çevir
img_to_tensor = transforms.ToTensor()
input = img_to_tensor(y).unsqueeze(0)  # (1, 1, H, W)

# 🖥️ Cihaz seçimi
device = torch.device("cuda:0" if (torch.cuda.is_available() and args.cuda) else "cpu")
print(f"Kullanılan cihaz: {device}")

# 🔁 Modeli yükle ve cihaza taşı
model = torch.load(args.model).to(device)
model.eval()
input = input.to(device)

# 📈 Model çıktısı al
with torch.no_grad():
    out = model(input)

# 📤 Çıktıyı tekrar görsele dönüştür
out = out.cpu()
out_img_y = out[0].detach().numpy()
out_img_y *= 255.0
out_img_y = out_img_y.clip(0, 255)
out_img_y = Image.fromarray(np.uint8(out_img_y[0]), mode='L')

# 🔀 YCbCr olarak birleştir ve RGB’ye dönüştür
out_img = Image.merge('YCbCr', [out_img_y, cb, cr]).convert('RGB')

# 💾 Kaydet
output_name = "zoomed_" + os.path.basename(args.image)
out_img.save(output_name)
print(f"Sonuç kaydedildi: {output_name}")
